In [2]:
import numpy as np
import random
import networkx as nx
import pandas as pd
from sklearn.metrics import jaccard_score
from pandas.core.frame import DataFrame
import heapq
import time

In [3]:
data = pd.read_csv("data.csv")
MSH = pd.read_csv("MSH.csv")
selected_node = pd.read_csv("selected_node.csv")

In [4]:
Characteristic_Graph = nx.Graph()
for i in range(len(data)):
    Characteristic_Graph.add_edge(data['0'][i]-1,data['1'][i]-1)
Connection_Matrix=np.array(nx.adjacency_matrix(Characteristic_Graph).todense())
Verte = list(Characteristic_Graph.nodes)

In [5]:
def difference_less_than_one(a,b):
    return np.abs(a-b).sum()

In [6]:
def jaccard(A,B):
    bigcap = len(list(set(A) & set(B)))
    bigcup = len(list(set(A) | set(B)))
    jaccard = bigcap/bigcup
    return jaccard

In [7]:
def similarest_ten(node,Verte):
    global Characteristic_Graph
    max_index = []
    Real_Jaccard = pd.DataFrame(columns=['node','jaccard'],dtype=float)
    for i in Verte:
        df_1 = pd.DataFrame([[i,jaccard(list(nx.all_neighbors(Characteristic_Graph, node)),list(nx.all_neighbors(Characteristic_Graph, i)))]],columns=['node','jaccard'],dtype=float)
        Real_Jaccard = pd.concat([Real_Jaccard,df_1], ignore_index=True)
    Real_Jaccard=Real_Jaccard.sort_values(by="jaccard" , ascending=False)
    print("Jaccard")
    if len(Real_Jaccard)>0:
        print(Real_Jaccard)
    Real_Jaccard_list = np.array(Real_Jaccard).tolist()
    max_ten = 10
    if max_ten>len(Real_Jaccard_list):
        max_ten=len(Real_Jaccard_list)
    for i in range(max_ten):
        max_index.append(Real_Jaccard_list[i][0])
    if (len(max_index)<10):
        while (len(max_index)<10):
            max_index.append(-1)
    return max_index

In [8]:
def LSH(node,b,r,bound):# b个行条，每行r个哈希函数
    global MSH
    global G
    global Verte
    MSH_tobe_Verteilung = np.array(MSH.iloc[:, 0:b*r])
    volunteer = []
    MSH_Verteilung = np.array_split(MSH_tobe_Verteilung, b, axis=1)
    print("MSH_Verteilung")
    print(MSH_Verteilung[0][node])
    for i in range (len(MSH_Verteilung)):
        for j in range (len(Verte)):
            if (difference_less_than_one(MSH_Verteilung[i][j],MSH_Verteilung[i][node])<=bound) & (j!=node) & (j not in volunteer):
                volunteer.append(j)
    print("volunteer")
    print(volunteer)
    return similarest_ten(node,volunteer)

In [9]:
test_node = []
for i in range (5):
    test_node+=[int(selected_node['node'][i])]

In [10]:
def conduct(b,r,bound):
    global test_node
    global Verte
    df = pd.DataFrame(columns=['node','accuracy','time','blank_control'],dtype=float)
    for i in range (5):
        print("-------------------")
        print("i")
        print(i)
        t_a=time.perf_counter()
        A = LSH(test_node[i],b,r,bound)
        t_b=time.perf_counter()
        Verte_without_i = list(Characteristic_Graph.nodes)
        if test_node[i] in Verte_without_i:
            Verte_without_i.remove(test_node[i])
        t_c=time.perf_counter()
        B = similarest_ten(test_node[i],Verte_without_i)
        t_d=time.perf_counter()
        accuracy = len(list(set(A) & set(B)))/len(B)
        run_time = t_b-t_a
        compare_time = t_d-t_c
        df1 = pd.DataFrame([[test_node[i],accuracy,run_time,compare_time]],columns=['node','accuracy','time','blank_control'],dtype=float)
        df = pd.concat([df,df1], ignore_index=True)
        print("A")
        print(A)
        print("B")
        print(B)
    print(df)

In [11]:
conduct(3128,4,1)

-------------------
i
0
MSH_Verteilung
[12866.   158.  5548.  8645.]
volunteer
[1616, 1629, 1342, 1623, 1593, 1611, 781, 1606, 1612, 1600, 2470, 1352, 1216, 1267, 1286, 1290, 1296, 1304, 1320, 1322, 1324, 1326, 1329, 1330, 1335, 1340, 1346, 1351, 1355, 1375, 1369, 1282, 1285, 1350, 2468, 10485, 10486, 10487, 10488, 10489, 10490, 240, 765, 791, 803, 1179, 1279, 1319, 1338, 1343, 767, 772, 1610, 1200, 1276, 214, 239, 776, 801, 222, 804, 1303, 1266, 1278, 1832, 777, 800, 763, 10492, 1273]
Jaccard
       node   jaccard
0    1616.0  0.837209
4    1593.0  0.787234
1    1629.0  0.731707
8    1612.0  0.705882
3    1623.0  0.701754
..      ...       ...
66    800.0  0.207547
67    763.0  0.169811
69   1273.0  0.142857
64   1832.0  0.113208
68  10492.0  0.066667

[70 rows x 2 columns]
Jaccard
          node   jaccard
4518    1616.0  0.837209
4515    1593.0  0.787234
4520    1629.0  0.731707
4517    1612.0  0.705882
4519    1623.0  0.701754
...        ...       ...
6536    3003.0  0.000000
6537  

In [12]:
conduct(1564,8,1)

-------------------
i
0
MSH_Verteilung
[12866.   158.  5548.  8645.  3260.    73.  1627.   860.]
volunteer
[1616, 1629, 1593, 1611, 1623, 1352, 1600, 1612, 781, 1282, 1343, 1200, 1319, 1610, 10486, 767, 1216, 1279, 1286, 1290, 1296, 1320, 1326, 1338, 1340, 1351, 2470, 2468, 240, 777, 1285, 1304, 1342, 1335, 804, 10488, 1375, 10485, 10487, 10489, 10490, 1267, 1329, 1355, 1350, 1346, 1322, 791, 803, 801, 1324, 1369, 772, 776, 239, 765, 1606, 1179, 1276, 214]
Jaccard
       node   jaccard
0    1616.0  0.837209
2    1593.0  0.787234
1    1629.0  0.731707
7    1612.0  0.705882
4    1623.0  0.701754
6    1600.0  0.655738
22   1326.0  0.645161
3    1611.0  0.642857
5    1352.0  0.642857
19   1290.0  0.606061
31   1304.0  0.606061
25   1351.0  0.606061
20   1296.0  0.606061
24   1340.0  0.597015
21   1320.0  0.597015
36   1375.0  0.584906
17   1279.0  0.579710
18   1286.0  0.579710
23   1338.0  0.579710
27   2468.0  0.574468
12   1319.0  0.571429
41   1267.0  0.566038
43   1355.0  0.563636
45 

In [11]:
conduct(783,16,1)

-------------------
i
0
MSH_Verteilung
[12866.   158.  5548.  8645.  3260.    73.  1627.   860.  4316.  5319.
   190.   663.  3189.   917.  1595.  4102.]
volunteer
[1616, 1629, 1352, 1593, 1612, 1623]
Jaccard
     node   jaccard
0  1616.0  0.837209
3  1593.0  0.787234
1  1629.0  0.731707
4  1612.0  0.705882
5  1623.0  0.701754
2  1352.0  0.642857
Jaccard
          node   jaccard
4518    1616.0  0.837209
4515    1593.0  0.787234
4520    1629.0  0.731707
4517    1612.0  0.705882
4519    1623.0  0.701754
...        ...       ...
6536    3003.0  0.000000
6537    3123.0  0.000000
6538    3134.0  0.000000
6539    9081.0  0.000000
18769  18770.0  0.000000

[18770 rows x 2 columns]
A
[1616.0, 1593.0, 1629.0, 1612.0, 1623.0, 1352.0, -1, -1, -1, -1]
B
[1616.0, 1593.0, 1629.0, 1612.0, 1623.0, 1600.0, 1326.0, 1352.0, 1611.0, 1351.0]
-------------------
i
1
MSH_Verteilung
[ 319. 1218. 1001.   14.  585.  169.  405.  251.   97. 1430.   44.    8.
  529.   57.  231.  157.]
volunteer
[]
Jaccard
Jaccard


In [12]:
conduct(391,32,1)

-------------------
i
0
MSH_Verteilung
[12866.   158.  5548.  8645.  3260.    73.  1627.   860.  4316.  5319.
   190.   663.  3189.   917.  1595.  4102.  1032.   297.   305.  4591.
   685.    48.  7370.  4862.  2328.    55.  8735.   161.  1823.  2628.
  3029.   636.]
volunteer
[1629, 1593, 1616]
Jaccard
     node   jaccard
2  1616.0  0.837209
1  1593.0  0.787234
0  1629.0  0.731707
Jaccard
          node   jaccard
4518    1616.0  0.837209
4515    1593.0  0.787234
4520    1629.0  0.731707
4517    1612.0  0.705882
4519    1623.0  0.701754
...        ...       ...
6536    3003.0  0.000000
6537    3123.0  0.000000
6538    3134.0  0.000000
6539    9081.0  0.000000
18769  18770.0  0.000000

[18770 rows x 2 columns]
A
[1616.0, 1593.0, 1629.0, -1, -1, -1, -1, -1, -1, -1]
B
[1616.0, 1593.0, 1629.0, 1612.0, 1623.0, 1600.0, 1326.0, 1352.0, 1611.0, 1351.0]
-------------------
i
1
MSH_Verteilung
[ 319. 1218. 1001.   14.  585.  169.  405.  251.   97. 1430.   44.    8.
  529.   57.  231.  157.  302. 

In [ ]:
conduct(3128,4,10)

volunteer
[1616, 1629, 1342, 1623, 1593, 1611, 781, 1606, 1612, 1600, 2470, 1352, 1216, 1267, 1286, 1290, 1296, 1304, 1320, 1322, 1324, 1326, 1329, 1330, 1335, 1340, 1346, 1351, 1355, 1375, 1369, 1282, 1285, 1350, 2468, 10485, 10486, 10487, 10488, 10489, 10490, 240, 765, 791, 803, 1179, 1279, 1319, 1338, 1343, 767, 772, 777, 1610, 1200, 1276, 214, 239, 776, 801, 222, 804, 1303, 1266, 1278, 1832, 1883, 800, 763, 1273, 10377, 808, 247, 10492, 1155]
Jaccard
       node   jaccard
0    1616.0  0.837209
4    1593.0  0.787234
1    1629.0  0.731707
8    1612.0  0.705882
3    1623.0  0.701754
..      ...       ...
66   1883.0  0.075949
73  10492.0  0.066667
72    247.0  0.061856
74   1155.0  0.053763
71    808.0  0.032787

[75 rows x 2 columns]
Jaccard
          node   jaccard
4518    1616.0  0.837209
4515    1593.0  0.787234
4520    1629.0  0.731707
4517    1612.0  0.705882
4519    1623.0  0.701754
...        ...       ...
6536    3003.0  0.000000
6537    3123.0  0.000000
6538    3134.0  0.000

In [13]:
conduct(1564,8,10)

-------------------
i
0
MSH_Verteilung
[12866.   158.  5548.  8645.  3260.    73.  1627.   860.]
volunteer
[1616, 1629, 1593, 1611, 1623, 1352, 1600, 1612, 781, 1282, 1343, 791, 803, 1200, 1267, 1319, 1335, 1375, 1610, 10486, 767, 1216, 1279, 1286, 1290, 1296, 1320, 1326, 1338, 1340, 1351, 2470, 2468, 240, 777, 1285, 1304, 1342, 1329, 804, 10488, 10485, 10487, 10489, 10490, 1346, 1355, 1350, 1303, 1322, 801, 1324, 1369, 772, 776, 239, 765, 1606, 1179, 1276, 214]
Jaccard
      node   jaccard
0   1616.0  0.837209
2   1593.0  0.787234
1   1629.0  0.731707
7   1612.0  0.705882
4   1623.0  0.701754
..     ...       ...
59  1276.0  0.400000
57  1606.0  0.395604
10  1343.0  0.391304
58  1179.0  0.357143
60   214.0  0.350877

[61 rows x 2 columns]
Jaccard
          node   jaccard
4518    1616.0  0.837209
4515    1593.0  0.787234
4520    1629.0  0.731707
4517    1612.0  0.705882
4519    1623.0  0.701754
...        ...       ...
6536    3003.0  0.000000
6537    3123.0  0.000000
6538    3134.0  0

In [14]:
conduct(783,16,10)

-------------------
i
0
MSH_Verteilung
[12866.   158.  5548.  8645.  3260.    73.  1627.   860.  4316.  5319.
   190.   663.  3189.   917.  1595.  4102.]
volunteer
[1616, 1629, 1352, 1593, 1612, 1623]
Jaccard
     node   jaccard
0  1616.0  0.837209
3  1593.0  0.787234
1  1629.0  0.731707
4  1612.0  0.705882
5  1623.0  0.701754
2  1352.0  0.642857
Jaccard
          node   jaccard
4518    1616.0  0.837209
4515    1593.0  0.787234
4520    1629.0  0.731707
4517    1612.0  0.705882
4519    1623.0  0.701754
...        ...       ...
6536    3003.0  0.000000
6537    3123.0  0.000000
6538    3134.0  0.000000
6539    9081.0  0.000000
18769  18770.0  0.000000

[18770 rows x 2 columns]
A
[1616.0, 1593.0, 1629.0, 1612.0, 1623.0, 1352.0, -1, -1, -1, -1]
B
[1616.0, 1593.0, 1629.0, 1612.0, 1623.0, 1600.0, 1326.0, 1352.0, 1611.0, 1351.0]
-------------------
i
1
MSH_Verteilung
[ 319. 1218. 1001.   14.  585.  169.  405.  251.   97. 1430.   44.    8.
  529.   57.  231.  157.]
volunteer
[]
Jaccard
Jaccard


In [15]:
conduct(391,32,10)

-------------------
i
0
MSH_Verteilung
[12866.   158.  5548.  8645.  3260.    73.  1627.   860.  4316.  5319.
   190.   663.  3189.   917.  1595.  4102.  1032.   297.   305.  4591.
   685.    48.  7370.  4862.  2328.    55.  8735.   161.  1823.  2628.
  3029.   636.]
volunteer
[1629, 1593, 1616]
Jaccard
     node   jaccard
2  1616.0  0.837209
1  1593.0  0.787234
0  1629.0  0.731707
Jaccard
          node   jaccard
4518    1616.0  0.837209
4515    1593.0  0.787234
4520    1629.0  0.731707
4517    1612.0  0.705882
4519    1623.0  0.701754
...        ...       ...
6536    3003.0  0.000000
6537    3123.0  0.000000
6538    3134.0  0.000000
6539    9081.0  0.000000
18769  18770.0  0.000000

[18770 rows x 2 columns]
A
[1616.0, 1593.0, 1629.0, -1, -1, -1, -1, -1, -1, -1]
B
[1616.0, 1593.0, 1629.0, 1612.0, 1623.0, 1600.0, 1326.0, 1352.0, 1611.0, 1351.0]
-------------------
i
1
MSH_Verteilung
[ 319. 1218. 1001.   14.  585.  169.  405.  251.   97. 1430.   44.    8.
  529.   57.  231.  157.  302. 

In [16]:
conduct(3128,4,100)

-------------------
i
0
MSH_Verteilung
[12866.   158.  5548.  8645.]
volunteer
[1352, 1616, 1629, 1162, 1216, 1285, 1342, 1593, 1267, 1329, 1375, 1612, 791, 803, 1279, 1290, 1296, 1303, 1304, 1320, 1326, 1338, 1340, 1346, 1351, 1623, 1355, 1611, 240, 767, 781, 1276, 1278, 1286, 1343, 1606, 6972, 763, 765, 776, 777, 801, 804, 1273, 1282, 1322, 1330, 1600, 2468, 2470, 3361, 1319, 10446, 1324, 1335, 1369, 214, 1350, 10485, 10486, 10487, 10488, 10489, 10490, 786, 1179, 772, 8658, 222, 239, 1200, 1297, 1610, 7749, 365, 4541, 5236, 5241, 6794, 903, 1026, 1037, 2046, 3007, 3290, 589, 2563, 608, 631, 637, 3524, 1266, 4228, 578, 598, 638, 800, 1147, 1150, 1157, 1649, 1742, 2273, 3763, 241, 3089, 13837, 13842, 8817, 2338, 4444, 304, 407, 9709, 9715, 5148, 7406, 8896, 2548, 1111, 1420, 2471, 2474, 2478, 2479, 2480, 2482, 2483, 2484, 2492, 2493, 2499, 2505, 2506, 2507, 2511, 2512, 2513, 2517, 2518, 2519, 2523, 3036, 5731, 5733, 5741, 5747, 5749, 5750, 5751, 586, 2549, 5556, 3405, 7457, 13959, 1396

In [17]:
conduct(1564,8,100)

-------------------
i
0
MSH_Verteilung
[12866.   158.  5548.  8645.  3260.    73.  1627.   860.]
volunteer
[1352, 1616, 1629, 1593, 1611, 1623, 1600, 1369, 1612, 1279, 1290, 1296, 1304, 1320, 1326, 1338, 1340, 1342, 1351, 1330, 781, 767, 1286, 1282, 1343, 791, 803, 804, 1200, 1267, 1319, 1329, 1335, 1375, 2468, 2470, 1610, 10485, 10486, 1216, 1324, 1606, 240, 777, 1285, 1346, 10487, 10489, 10490, 10488, 772, 1355, 801, 1350, 214, 239, 776, 1322, 1303, 1179, 765, 1276, 1266, 1278, 222]
Jaccard
      node   jaccard
1   1616.0  0.837209
3   1593.0  0.787234
2   1629.0  0.731707
8   1612.0  0.705882
5   1623.0  0.701754
..     ...       ...
59  1179.0  0.357143
54   214.0  0.350877
63  1278.0  0.341880
62  1266.0  0.320000
64   222.0  0.310078

[65 rows x 2 columns]
Jaccard
          node   jaccard
4518    1616.0  0.837209
4515    1593.0  0.787234
4520    1629.0  0.731707
4517    1612.0  0.705882
4519    1623.0  0.701754
...        ...       ...
6536    3003.0  0.000000
6537    3123.0  0.0

In [18]:
conduct(783,16,100)

-------------------
i
0
MSH_Verteilung
[12866.   158.  5548.  8645.  3260.    73.  1627.   860.  4316.  5319.
   190.   663.  3189.   917.  1595.  4102.]
volunteer
[1616, 1629, 1593, 1623, 1612, 1611, 1352, 1600, 1267, 1375, 791, 803, 1282]
Jaccard
      node   jaccard
0   1616.0  0.837209
2   1593.0  0.787234
1   1629.0  0.731707
4   1612.0  0.705882
3   1623.0  0.701754
7   1600.0  0.655738
5   1611.0  0.642857
6   1352.0  0.642857
9   1375.0  0.584906
8   1267.0  0.566038
10   791.0  0.524590
11   803.0  0.524590
12  1282.0  0.490566
Jaccard
          node   jaccard
4518    1616.0  0.837209
4515    1593.0  0.787234
4520    1629.0  0.731707
4517    1612.0  0.705882
4519    1623.0  0.701754
...        ...       ...
6536    3003.0  0.000000
6537    3123.0  0.000000
6538    3134.0  0.000000
6539    9081.0  0.000000
18769  18770.0  0.000000

[18770 rows x 2 columns]
A
[1616.0, 1593.0, 1629.0, 1612.0, 1623.0, 1600.0, 1611.0, 1352.0, 1375.0, 1267.0]
B
[1616.0, 1593.0, 1629.0, 1612.0, 1623.

In [19]:
conduct(391,32,100)

-------------------
i
0
MSH_Verteilung
[12866.   158.  5548.  8645.  3260.    73.  1627.   860.  4316.  5319.
   190.   663.  3189.   917.  1595.  4102.  1032.   297.   305.  4591.
   685.    48.  7370.  4862.  2328.    55.  8735.   161.  1823.  2628.
  3029.   636.]
volunteer
[1629, 1593, 1616]
Jaccard
     node   jaccard
2  1616.0  0.837209
1  1593.0  0.787234
0  1629.0  0.731707
Jaccard
          node   jaccard
4518    1616.0  0.837209
4515    1593.0  0.787234
4520    1629.0  0.731707
4517    1612.0  0.705882
4519    1623.0  0.701754
...        ...       ...
6536    3003.0  0.000000
6537    3123.0  0.000000
6538    3134.0  0.000000
6539    9081.0  0.000000
18769  18770.0  0.000000

[18770 rows x 2 columns]
A
[1616.0, 1593.0, 1629.0, -1, -1, -1, -1, -1, -1, -1]
B
[1616.0, 1593.0, 1629.0, 1612.0, 1623.0, 1600.0, 1326.0, 1352.0, 1611.0, 1351.0]
-------------------
i
1
MSH_Verteilung
[ 319. 1218. 1001.   14.  585.  169.  405.  251.   97. 1430.   44.    8.
  529.   57.  231.  157.  302. 